In [1]:
import warnings
from dataclasses import dataclass
from pathlib import Path

import cv2
import joblib
import mahotas as mh
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage as ski
import torch
from imblearn.over_sampling import RandomOverSampler
from skimage import img_as_ubyte
from skimage.feature import graycomatrix, graycoprops
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tqdm.auto import tqdm
from transformers import ViTImageProcessor, ViTModel
from xgboost import XGBClassifier

In [2]:
DATASET_NAME = "drsprg"
DATA_BASE_DIR = Path(f"../data/processed/{DATASET_NAME}/")
PREP_STUDIES = DATA_BASE_DIR / Path("artifacts/prep_studies.pkl")

# Experiment variables
SEED = 42
CV = 5

In [3]:
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

In [4]:
tqdm.pandas()

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
studies = tuple(joblib.load(PREP_STUDIES))

In [7]:
# img = studies[0][0][0][0]

# image = img
# extract_zernike_moments(image)

In [8]:
class ZernikeMomentsEncoder(BaseEstimator, TransformerMixin):
    """
    Extracts Zernike Moments from a binary or grayscale image.

    Attributes:
    ----------
    radius : int, optional
        The radius of the circle to compute the moments (default is 21).
    degree : int, optional
        The maximum degree for Zernike moments (default is 8).
    """

    def __init__(self, radius: int = 21, degree: int = 8) -> None:
        self.radius = radius
        self.degree = degree

    def transform(self, X, y=None) -> np.ndarray:
        X = list(X)
        X = np.array([self._extract(x) for x in X])
        return X

    def fit(self, X, y=None):
        return self

    def _extract(self, image: np.ndarray) -> np.ndarray:

        # Gray image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Ensure the image is binary or grayscale
        if len(image.shape) > 2:
            raise ValueError("Input image should be 2D (grayscale or binary).")

        # Normalize the image if it's not binary (convert it to binary)
        binary_image = (image > image.mean()).astype(np.uint8)

        # Calculate Zernike Moments using the mahotas library
        moments = mh.features.zernike_moments(binary_image, self.radius, self.degree)

        return moments


class GLCMEncoder(BaseEstimator, TransformerMixin):
    """
    A class to represent an GLCM encoder.

    Atributes:
    - distances: List of pixel pair distance offsets for GLCM computation.
    - angles: List of angles in radians for GLCM computation.
    - levels: The number of gray levels for the GLCM (default is 256).
    - symmetric: If True, GLCM matrix will be symmetric.
    - normed: If True, the GLCM matrix will be normalized.

    Methods:
    - transform(): Extracts the GLCM features (contrast, correlation, energy, homogeneity) from an image.
    """

    def __init__(
        self,
        distances: list[int] = [1],
        angles: list[int] = [0],
        levels: int = 256,
        symmetric: bool = True,
        normed: bool = True,
    ):
        self.distances = distances
        self.angles = angles
        self.levels = levels
        self.symmetric = symmetric
        self.normed = normed

    def transform(self, X, y=None) -> np.ndarray:
        X = list(X)
        X = np.array([self._extract(x) for x in X])
        return X

    def fit(self, X, y=None):
        return self

    def _extract(self, image: np.ndarray) -> np.ndarray:

        # Gray image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Convert image to 8-bit unsigned integer type, if needed
        image = img_as_ubyte(image)

        # Compute the GLCM matrix
        glcm = graycomatrix(
            image,
            distances=self.distances,
            angles=self.angles,
            levels=self.levels,
            symmetric=self.symmetric,
            normed=self.normed,
        )

        # Extract GLCM properties (features)
        contrast = graycoprops(glcm, "contrast").mean()
        correlation = graycoprops(glcm, "correlation").mean()
        energy = graycoprops(glcm, "energy").mean()
        homogeneity = graycoprops(glcm, "homogeneity").mean()

        # Return the features as a dictionary
        features = np.array([contrast, correlation, energy, homogeneity])

        return features


class LBPEncoder(BaseEstimator, TransformerMixin):
    """LBP encoder for image data."""

    def __init__(self, radius: int = 1, sampling_pixels: int = 106):
        self.radius = radius
        self.sampling_pixels = sampling_pixels

    def transform(self, X, y=None):
        """Extract the LBP from the images batch."""
        X = list(X)
        cvt_imgs = [self._cvt(img) for img in X]
        imgs_lbps = [self._get_lbp(img) for img in cvt_imgs]
        imgs_hists = [self._get_hist(img_lbp) for img_lbp in imgs_lbps]
        features = self._get_features(imgs_hists)
        return features

    def fit(self, X, y=None):
        return self

    def _cvt(self, img):
        if isinstance(img, float):
            print("test")
        if len(img.shape) > 2:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        i_min = np.min(img)
        i_max = np.max(img)
        if i_max - i_min != 0:
            img = (img - i_min) / (i_max - i_min)

        img = img.astype(np.uint8)

        return img

    def _get_lbp(self, img):
        lbp = ski.feature.local_binary_pattern(
            img, self.sampling_pixels, self.radius, method="uniform"
        )
        return (img, lbp)

    def _get_hist(self, img_lbp):
        img, lbp = img_lbp
        hist, _ = np.histogram(
            lbp.ravel(),
            bins=np.arange(0, self.sampling_pixels + 3),
            range=(0, self.sampling_pixels + 2),
        )
        hist = hist.astype("float")
        hist /= hist.sum() + 1e-6
        return img, hist

    def _get_features(self, imgs_hists):
        hists = [img_hist[1] for img_hist in imgs_hists]
        features = []
        for h in hists:
            features.extend(h)
        return hists


class ViTEncoder(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        device,
        model_name="google/vit-base-patch16-224",
        image_size=128,
    ):
        self.model_name = model_name
        self.device = device
        self.feature_extractor = ViTImageProcessor.from_pretrained(model_name)
        self.model = ViTModel.from_pretrained(model_name, add_pooling_layer=False).to(
            device
        )
        self.image_size = image_size

    def transform(self, X, y=None):
        """Extract features from images using ViT"""

        X = list(X)

        if not isinstance(X, list):
            raise ValueError("Input is not a list of images.")
        elif not all(isinstance(x, np.ndarray) for x in X):
            raise ValueError("Not all instances are numpy arrays (images).")

        inputs = self.feature_extractor(X, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state[:, 0, :].cpu().numpy()

    def fit(self, X, y=None):
        return self

In [9]:
samples = [(([pi[0] for pi in study[0]]), study[1]) for study in tqdm(studies, desc="Studies")]
df = pd.DataFrame(data=samples, columns=["features", "labels"])

Studies:   0%|          | 0/102 [00:00<?, ?it/s]

In [10]:
df["features"] = df["features"].progress_apply(lambda images: np.mean(np.stack(images, axis=0), axis=0).astype(np.uint8))

  0%|          | 0/102 [00:00<?, ?it/s]

In [11]:
@dataclass
class CVResult:
    algo: str
    encoder: str
    test_acc: float
    test_macro_prec: float
    test_weighted_prec: float
    test_macro_recall: float
    test_weighted_recall: float
    test_macro_f1: float
    test_weighted_f1: float


def _create_lbp_pipeline(clf: BaseEstimator) -> Pipeline:
    pipeline = Pipeline(
        [
            ("encoder", LBPEncoder()),
            ("clf", clf),
        ]
    )
    return pipeline


def _create_vit_pipeline(clf: BaseEstimator) -> Pipeline:
    pipeline = Pipeline([("encoder", ViTEncoder(device=device)), ("clf", clf)])
    return pipeline


def _create_glcm_pipeline(clf: BaseEstimator) -> Pipeline:
    pipeline = Pipeline([("encoder", GLCMEncoder()), ("clf", clf)])
    return pipeline


def _create_zernike_moments_pipeline(clf: BaseEstimator) -> Pipeline:
    pipeline = Pipeline([("encoder", ZernikeMomentsEncoder()), ("clf", clf)])
    return pipeline


def _run_sklearn_cv(
    clf: BaseEstimator | Pipeline,
    X: np.ndarray,
    y: np.ndarray,
    cv: int,
    random_state,
    shuffle=True,
):
    """Run the experiments using cross-validation."""

    ALGO_NAME_DICT = {
        ExtraTreesClassifier: "Extra Trees",
        DecisionTreeClassifier: "Decision Tree",
        SVC: "Support Vector",
        RandomForestClassifier: "Random Forest",
        XGBClassifier: "XGBoost",
        MLPClassifier: "MLP",
    }

    ENCODER_NAME_DICT = {
        LBPEncoder: "LBP",
        ViTEncoder: "ViT",
        GLCMEncoder: "GLCM",
        ZernikeMomentsEncoder: "ZM"
    }

    # Generate split
    cv_results = []
    skf = StratifiedKFold(n_splits=cv, random_state=random_state, shuffle=shuffle)
    for train_index, test_index in skf.split(np.array(X), y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Resample the training data
        X_train, y_train = RandomOverSampler(random_state=random_state).fit_resample(
            pd.DataFrame(data=X_train), y_train
        )
        X_train = np.squeeze(X_train.to_numpy())

        # Train model
        clf.fit(X_train, y_train)

        # Evaluate model
        y_pred = clf.predict(X_test)
        cv_result = CVResult(
            algo=ALGO_NAME_DICT[type(clf.named_steps["clf"])],
            encoder=ENCODER_NAME_DICT[type(clf.named_steps["encoder"])],
            test_acc=accuracy_score(y_test, y_pred),
            test_macro_prec=precision_score(y_test, y_pred, average="macro"),
            test_weighted_prec=precision_score(y_test, y_pred, average="weighted"),
            test_macro_recall=recall_score(y_test, y_pred, average="macro"),
            test_weighted_recall=recall_score(y_test, y_pred, average="weighted"),
            test_macro_f1=f1_score(y_test, y_pred, average="macro"),
            test_weighted_f1=f1_score(y_test, y_pred, average="weighted"),
        )
        cv_results.append(cv_result)

    return cv_results


def run_experiments(
    clfs: list[BaseEstimator],
    studies: pd.DataFrame,
    cv: int,
    random_state,
) -> dict[str, list[np.ndarray]]:
    """Run experiments."""

    label_encoder = LabelEncoder()
    studies["labels"] = label_encoder.fit_transform(studies["labels"])

    lbp_cv_results = [
        _run_sklearn_cv(
            _create_lbp_pipeline(clf),
            np.array(studies["features"]),
            np.array(studies["labels"]),
            cv=cv,
            random_state=random_state,
        )
        for clf in tqdm(clfs, desc="LBP")
    ]

    vit_cv_results = [
        _run_sklearn_cv(
            _create_vit_pipeline(clf),
            np.array(studies["features"]),
            np.array(studies["labels"]),
            cv=cv,
            random_state=random_state,
        )
        for clf in tqdm(clfs, desc="ViT")
    ]

    glcm_cv_results = [
        _run_sklearn_cv(
            _create_glcm_pipeline(clf),
            np.array(studies["features"]),
            np.array(studies["labels"]),
            cv=cv,
            random_state=random_state,
        )
        for clf in tqdm(clfs, desc="GLCM")
    ]
    
    zernike_moments_cv_results = [
        _run_sklearn_cv(
            _create_zernike_moments_pipeline(clf),
            np.array(studies["features"]),
            np.array(studies["labels"]),
            cv=cv,
            random_state=random_state,
        )
        for clf in tqdm(clfs, desc="ZM")
    ]

    cv_result_list = [lbp_cv_results, vit_cv_results, glcm_cv_results, zernike_moments_cv_results]

    return cv_result_list

In [12]:
clfs = [
    SVC(random_state=SEED),
    RandomForestClassifier(random_state=SEED),
    DecisionTreeClassifier(random_state=SEED),
    ExtraTreesClassifier(random_state=SEED),
    # XGBClassifier(
    #     objective="multi:softprob",
    #     eval_metric="mlogloss",
    # ),
    MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=SEED),
]

with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=UndefinedMetricWarning)
    warnings.simplefilter(action="ignore", category=UserWarning)
    warnings.simplefilter(action="ignore", category=FutureWarning)
    cv_results_list = run_experiments(clfs, df, cv=CV, random_state=SEED)

LBP:   0%|          | 0/5 [00:00<?, ?it/s]

ViT:   0%|          | 0/5 [00:00<?, ?it/s]

GLCM:   0%|          | 0/5 [00:00<?, ?it/s]

GLCM:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
def scores_to_df(cv_results_list: list[list[list[CVResult]]]) -> pd.DataFrame:
    """Transform the scores dictionary into a dataframe object."""
    data = []
    for results_group in cv_results_list:
        for cv_results in results_group:
            data.append(
                {
                    "algo": cv_results[0].algo,
                    "encoder": cv_results[0].encoder,
                    "acc": np.mean([cv_result.test_acc for cv_result in cv_results]),
                    "macro_prec": np.mean([cv_result.test_macro_prec for cv_result in cv_results]),
                    "weighted_prec": np.mean([cv_result.test_weighted_prec for cv_result in cv_results]),
                    "macro_recall": np.mean([cv_result.test_macro_recall for cv_result in cv_results]),
                    "weighted_recall": np.mean([cv_result.test_weighted_recall for cv_result in cv_results]),
                    "macro_f1": np.mean([cv_result.test_macro_f1 for cv_result in cv_results]),
                    "weighted_f1": np.mean([cv_result.test_weighted_f1 for cv_result in cv_results]),
                }
            )
    df = pd.DataFrame(data=data)
    return df

scores_to_df(cv_results_list)

,algo,encoder,acc,macro_prec,weighted_prec,macro_recall,weighted_recall,macro_f1,weighted_f1
0,Support Vector,LBP,0.707619,0.708901,0.712014,0.699747,0.707619,0.700581,0.706195
1,Random Forest,LBP,0.637619,0.650803,0.650510,0.624495,0.637619,0.625034,0.632801
2,Decision Tree,LBP,0.518095,0.519722,0.528147,0.517424,0.518095,0.513604,0.518145
3,Extra Trees,LBP,0.695714,0.699963,0.698331,0.677399,0.695714,0.679612,0.688707
4,MLP,LBP,0.431429,0.215714,0.186469,0.500000,0.431429,0.301281,0.260296
5,Support Vector,ViT,0.883810,0.888924,0.893847,0.885985,0.883810,0.882480,0.883919
6,Random Forest,ViT,0.854286,0.859582,0.858376,0.845707,0.854286,0.849006,0.852973
7,Decision Tree,ViT,0.803333,0.828999,0.826245,0.789394,0.803333,0.786868,0.794011
8,Extra Trees,ViT,0.824286,0.832967,0.829592,0.812374,0.824286,0.817684,0.822294
9,MLP,ViT,0.863810,0.869580,0.875795,0.867803,0.863810,0.862895,0.864098
